## Modelo Para Predecir Sector a partir de ref1

In [2]:
import sys
from pyspark.sql import *
import matplotlib as plt
import numpy as np
import pandas as pd
from optimus.optimus import Optimus
from datetime import datetime
from pyspark.sql.functions import col, asc
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.sql import *

In [3]:
op =Optimus()
tr = op.load.csv("/FileStore/tables/transacs_out.csv", header=None, encoding="Latin1")
tr = (tr
      .withColumnRenamed("_c0", "id_trn_ach")
      .withColumnRenamed("_c1", "id_cliente")
      .withColumnRenamed("_c2", "fechahora")
      .withColumnRenamed("_c3", "valor_trx")
      .withColumnRenamed("_c4", "ref1")
      .withColumnRenamed("_c5", "sector")
      .withColumnRenamed("_c6", "subsector")
      .withColumnRenamed("_c7", "descripcion"))
tr.dtypes

Out[2]: 
[('id_trn_ach', 'int'),
 ('id_cliente', 'int'),
 ('fechahora', 'bigint'),
 ('valor_trx', 'double'),
 ('ref1', 'string'),
 ('sector', 'string'),
 ('subsector', 'string'),
 ('descripcion', 'string')]

In [4]:
drop_list = ['id_cliente', 'fechahora', 'valor_trx', 'subsector', 'descripcion']
data = tr.select([column for column in tr.columns if column not in drop_list])
data.show(5)

+----------+--------------------+------+
id_trn_ach| ref1|sector|
+----------+--------------------+------+
 230435642| CC | null|
 222356110|Referencia: Cont...| null|
 309137749| CC | null|
 324614737| CC | null|
 235344690|MEDICINA PREPAGAD...| null|
+----------+--------------------+------+
only showing top 5 rows

## Matriz con datos de ref1 para predecir sector

In [6]:
#display(data.filter("ref1!=''"))
data1=data.cols.trim("*")
data1=data1.filter("ref1!=''")
display(data1)


id_trn_ach,ref1,sector
230435642,CC,null
222356110,Referencia: Contrato: Valor: CC,null
309137749,CC,null
324614737,CC,null
235344690,MEDICINA PREPAGADA COLSANITAS CE,null
320049316,RECAUDO COLSANITAS CC,null
230519178,CC,null
241307506,Pago de la Planilla Cesantias CEDULA DE CIUDADANIA,SERVICIOS FINANCIEROS
316193109,CC,null
282076350,Pago de la factura # CONJUNTO RESIDENCIAL PIETRA SANTA PH IDC,null


In [7]:
data1= data1.filter("sector is null")
#display(data1.filter("sector is null"))
display(data1)

id_trn_ach,ref1,sector
230435642,CC,null
222356110,Referencia: Contrato: Valor: CC,null
309137749,CC,null
324614737,CC,null
235344690,MEDICINA PREPAGADA COLSANITAS CE,null
320049316,RECAUDO COLSANITAS CC,null
230519178,CC,null
316193109,CC,null
282076350,Pago de la factura # CONJUNTO RESIDENCIAL PIETRA SANTA PH IDC,null
300800637,Pago de la factura # CONJUNTO RESIDENCIAL PIETRA SANTA PH IDC,null


##Entrenando el modelo

In [9]:
data=data.dropna()

In [10]:
data.groupBy("sector") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(5)
#sorted(gdf.agg({"*":"count"}).collect())

+--------------------+-------+
 sector| count|
+--------------------+-------+
MEDIOS DE COMUNIC...|1173421|
SERVICIOS FINANCI...|1061605|
 GOBIERNO| 519568|
 RECURSOS NATURALES| 442490|
SERVICIOS NO FINA...| 68965|
+--------------------+-------+
only showing top 5 rows

In [11]:
data.groupBy("ref1") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(5)

+--------------------+------+
 ref1| count|
+--------------------+------+
 Pago de Saldo |513976|
Empresas Publicas...|420465|
Transaccion_para_...|282528|
 CPV |276880|
Ref pago express ...|226809|
+--------------------+------+
only showing top 5 rows

In [12]:
display(data)

id_trn_ach,ref1,sector
241307506,Pago de la Planilla Cesantias CEDULA DE CIUDADANIA,SERVICIOS FINANCIEROS
359125394,CPV,SERVICIOS FINANCIEROS
285847659,Pago Factura Asociado BancoomevaPFA,SERVICIOS FINANCIEROS
319249942,Presentación y Pago del Impuesto Predial Unificado AAAXDFT,GOBIERNO
335337578,EDIF MIRABELL PH BOG,SERVICIOS FINANCIEROS
365833134,EDIF MIRABELL PH BOG,SERVICIOS FINANCIEROS
346095462,Pago de Saldo,MEDIOS DE COMUNICACION
228320905,Referencia de pago express No |,MEDIOS DE COMUNICACION
249117129,Referencia de pago express No |,MEDIOS DE COMUNICACION
278487871,Referencia pago express No | CC,MEDIOS DE COMUNICACION


##Procedimiento

Se va a utilizar el Modelo Pipeline: Spark Machine Learning Peipelines Api, el cual es similar a Scikit _ Learn. Este pipeline incluye tres pasos

#### 1. Tokenización: 
Utilizando la expresion regular `regexTokenizer`
####2. Quitar palabras Vacías
Se utiliza la expresion `stopwordsRemover` para quitar pronombres, articulos, preposiciones
####3. Contar Vectores
Mediante la expresión `countVectors`

In [15]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# tokenizar la expresión
regexTokenizer = RegexTokenizer(inputCol="ref1", outputCol="words", pattern="\\W")

# Quitar palabras vacias
add_stopwords = ["http","https","amp","rt","t","c","the"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# Conteo de palabras
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

Ahora utilizamos la expresión `StringIndexer` para codificar una columna de etiquetas en cadena en una columna de indices en cadena. Los indices se encuentran ordenados por frecuencias de etiqueta [0,numLabels], cuya etiqueta más frecuente tiene el índice 0.
La etiqueta de Sector se codificara con el fin de etiquetar indices , en este caso la etiqueta más frecuente es MEDIOS DE COMUNIOCACIÓN, por tanto se indexo como 0

In [17]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "sector", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# ajustar el pipeline a las columnas de entrenamiento
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
dataset.show(5)

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
id_trn_ach| ref1| sector| words| filtered| features|label|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
 241307506|Pago de la Planil...|SERVICIOS FINANCI...|[pago, de, la, pl...|[pago, de, la, pl...|(9213,[0,1,63,120...| 1.0|
 359125394| CPV |SERVICIOS FINANCI...| [cpv]| [cpv]| (9213,[13],[1.0])| 1.0|
 285847659|Pago Factura Asoc...|SERVICIOS FINANCI...|[pago, factura, a...|[pago, factura, a...|(9213,[1,27,30,31...| 1.0|
 319249942|Presentación y Pa...| GOBIERNO|[presentaci, n, y...|[presentaci, n, y...|(9213,[1,19,21,22...| 2.0|
 335337578|EDIF MIRABELL PH ...|SERVICIOS FINANCI...|[edif, mirabell, ...|[edif, mirabell, ...|(9213,[87,99,164,...| 1.0|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows

### Partición de los datos
Se realiza la partición de los datos, donde se dispone del 70% para entrenamiento y 30% para prueba

In [19]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 2313196
Test Dataset Count: 992359

### Entrenamiento del Modelo
Se entreno el modelo de regresión logistica utilizando caracteristicas de un vector de conteo, el cual realizara prediciiones y registros en el conjunto de pruebas, luego se observa las 10 mejores predicciones con la probabilidad más alta

In [21]:
#Logistic Regression using Count Vector Features

lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("id_trn_ach","ref1","sector","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+----------+------------------------------+----------------------+------------------------------+-----+----------+
id_trn_ach| ref1| sector| probability|label|prediction|
+----------+------------------------------+----------------------+------------------------------+-----+----------+
 335558758|ETC Aviso: Variedades Maqu...|MEDIOS DE COMUNICACION|[0.9331403764181024,0.03544...| 0.0| 0.0|
 354854609|ETC Aviso: Empleos Estándar...|MEDIOS DE COMUNICACION|[0.9208218823150699,0.03675...| 0.0| 0.0|
 305059196|ETC Aviso: Empleos Estándar...|MEDIOS DE COMUNICACION|[0.9208218823150699,0.03675...| 0.0| 0.0|
 315581096|ETC Aviso: Empleos Estándar...|MEDIOS DE COMUNICACION|[0.9208218823150699,0.03675...| 0.0| 0.0|
 283743163|ETC Aviso: Empleos Estándar...|MEDIOS DE COMUNICACION|[0.9208218823150699,0.03675...| 0.0| 0.0|
 310976897| Ref pago express No | CEL |MEDIOS DE COMUNICACION|[0.9161036016097711,0.03817...| 0.0| 0.0|
 313836022| Ref pago express No | CEL |MEDIOS DE COMUNICACION|[0.9161036016097711,0.03817...| 0.0| 0.0|
 266118063|Referencia pago express No ...|MEDIOS DE COMUNICACION|[0.9144088611674521,0.03901...| 0.0| 0.0|
 274468901|Referencia pago express No ...|MEDIOS DE COMUNICACION|[0.9144088611674521,0.03901...| 0.0| 0.0|
 365242354|Referencia pago APP No B | PP |MEDIOS DE COMUNICACION|[0.910501132742972,0.039665...| 0.0| 0.0|
+----------+------------------------------+----------------------+------------------------------+-----+----------+
only showing top 10 rows

In [22]:
display(predictions)

id_trn_ach,ref1,sector,words,filtered,features,label,rawPrediction,probability,prediction
215122439,Obligaciones seleccionadas por el usuario (ref: ) CC,GOBIERNO,"List(obligaciones, seleccionadas, por, el, usuario, ref, cc)","List(obligaciones, seleccionadas, por, el, usuario, ref, cc)","List(0, 9213, List(2, 14, 25, 78, 91, 148, 216), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",2.0,"List(1, 10, List(), List(2.772971257714097, 1.9292877644396234, 3.9453614796352356, 1.1447849576150724, 1.480258982548745, -0.5047783341987545, -2.1227208655254914, -2.4274166807716346, -2.516254555452494, -3.7014940060070733))","List(1, 10, List(), List(0.19275506988881447, 0.08290843630064268, 0.6225416519999775, 0.037834977858098855, 0.05291612113846975, 0.007269377944087386, 0.001441563360631867, 0.001062933327730203, 9.725775165377725E-4, 2.972906650096677E-4))",2.0
215123596,CPV,SERVICIOS FINANCIEROS,List(cpv),List(cpv),"List(0, 9213, List(13), List(1.0))",1.0,"List(1, 10, List(), List(2.707435116390643, 4.425638154388275, 2.1311655822453117, 1.425778128857572, 0.4665611953784687, -0.43543035456731255, -2.0994271228947863, -2.416956892239868, -2.5059419215054044, -3.698821886052687))","List(1, 10, List(), List(0.13184241659648624, 0.734955853216441, 0.07409424482558839, 0.03659641951735145, 0.01402348570908886, 0.005690180243072984, 0.0010776094940865828, 7.844404539549308E-4, 7.176526207398034E-4, 2.1769732318999596E-4))",1.0
215123613,CONJ RES GUALI PH,SERVICIOS FINANCIEROS,"List(conj, res, guali, ph)","List(conj, res, guali, ph)","List(0, 9213, List(64, 76, 99, 506), List(1.0, 1.0, 1.0, 1.0))",1.0,"List(1, 10, List(), List(2.426723736962087, 5.055525867313431, 1.9441632061927026, 1.3048693887059564, 0.4646012418904374, -0.4618596953546766, -2.1043160893635724, -2.4205484903226835, -2.5093279998801523, -3.699831166143529))","List(1, 10, List(), List(0.06240546275433719, 0.8647625374246434, 0.038516714306496426, 0.020323918982519402, 0.008771697283997648, 0.0034731757716057713, 6.720738755534151E-4, 4.898679511713473E-4, 4.482523512290557E-4, 1.3629929844630607E-4))",1.0
215123633,Referencia de pago express No | CC,MEDIOS DE COMUNICACION,"List(referencia, de, pago, express, no, cc)","List(referencia, de, pago, express, no, cc)","List(0, 9213, List(0, 1, 2, 4, 6, 17), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",0.0,"List(1, 10, List(), List(5.276380183576645, 2.164739725866507, 1.6324192723511397, 1.5790735951612822, 0.5128428350973939, -0.4304398539808395, -2.0973542878610267, -2.42192799811386, -2.5155280260893553, -3.7002054460063345))","List(1, 10, List(), List(0.9017573907661409, 0.0401533172365928, 0.023579663031026356, 0.022354752255984375, 0.00769682700990082, 0.0029967413397441454, 5.65871105414968E-4, 4.0903168114999033E-4, 3.724834417269445E-4, 1.1392213231883343E-4))",0.0
215125608,Recarga express,MEDIOS DE COMUNICACION,"List(recarga, express)","List(recarga, express)","List(0, 9213, List(4, 10), List(1.0, 1.0))",0.0,"List(1, 10, List(), List(4.02962481475416, 3.272833476492395, 2.062518335750137, 1.3516157993859423, 0.4575634497424687, -0.44684460491824357, -2.1018411565687383, -2.418699746749999, -2.5074586238807166, -3.699311744004273))","List(1, 10, List(), List(0.580434683465461, 0.2723222040155586, 0.08118015316049301, 0.03987574569409268, 0.016308980582584492, 0.006601572362614341, 0.0012615123837884609, 9.189282033683964E-4, 8.40880130271514E-4, 2.5534000176750466E-4))",0.0
215126063,Referencia de pago express No | CC,MEDIOS DE COMUNICACION,"List(referencia, de, pago, express, no, cc)","List(referencia, de, pago, express, no, cc)","List(0, 9213, List(0, 1, 2, 4, 6, 17), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",0.0,"List(1, 10, List(), List(5.276380183576645, 2.164739725866507, 1.6324192723511397, 1.5790735951612822, 0.5128428350973939, -0.4304398539808395, -2.0973542878610267, -2.42192799811386, -2.5155280260893553, -3.7002054460063345))","List(1, 10, List(), List(0.9017573907661409, 0.0401533172365928, 0.023579663031026356, 0.0223547

##Evaluación del Modelo

In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[15]: 0.9807099584838458

#### Predicción en la matriz data1, es decir para los datos cuyo sector no esta definido pero tenemos  ref1

In [26]:
display(data1)

id_trn_ach,ref1,sector
230435642,CC,null
222356110,Referencia: Contrato: Valor: CC,null
309137749,CC,null
324614737,CC,null
235344690,MEDICINA PREPAGADA COLSANITAS CE,null
320049316,RECAUDO COLSANITAS CC,null
230519178,CC,null
316193109,CC,null
282076350,Pago de la factura # CONJUNTO RESIDENCIAL PIETRA SANTA PH IDC,null
300800637,Pago de la factura # CONJUNTO RESIDENCIAL PIETRA SANTA PH IDC,null


In [27]:
label_stringIdx = StringIndexer(inputCol = "sector", outputCol = "label")

pipeline1 = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors])
pipelineFit1 = pipeline1.fit(data1)
dataset1 = pipelineFit1.transform(data1)

In [28]:
dataset1 = dataset1.drop("sector")

In [29]:
dataset1.show()

+----------+--------------------+--------------------+--------------------+--------------------+
id_trn_ach| ref1| words| filtered| features|
+----------+--------------------+--------------------+--------------------+--------------------+
 230435642| CC| [cc]| [cc]| (10000,[0],[1.0])|
 222356110|Referencia: Cont...|[referencia, cont...|[referencia, cont...|(10000,[0,12,15,1...|
 309137749| CC| [cc]| [cc]| (10000,[0],[1.0])|
 324614737| CC| [cc]| [cc]| (10000,[0],[1.0])|
 235344690|MEDICINA PREPAGAD...|[medicina, prepag...|[medicina, prepag...|(10000,[107,130,1...|
 320049316|RECAUDO COLSANITA...|[recaudo, colsani...|[recaudo, colsani...|(10000,[0,33,130]...|
 230519178| CC| [cc]| [cc]| (10000,[0],[1.0])|
 316193109| CC| [cc]| [cc]| (10000,[0],[1.0])|
 282076350|Pago de la factur...|[pago, de, la, fa...|[pago, de, la, fa...|(10000,[1,2,3,7,5...|
 300800637|Pago de la factur...|[pago, de, la, fa...|[pago, de, la, fa...|(10000,[1,2,3,7,5...|
 239292415|Pago de saldo pla...|[pago, de, saldo,...|[pago, de, saldo,...|(10000,[1,2,89,10...|
 244472692|Pago de saldo pla...|[pago, de, saldo,...|[pago, de, saldo,...|(10000,[1,2,89,10...|
 271391421| CC| [cc]| [cc]| (10000,[0],[1.0])|
 350884991| CC| [cc]| [cc]| (10000,[0],[1.0])|
 227400411|Pago por iRecaudo...|[pago, por, ireca...|[pago, por, ireca...|(10000,[1,2,13,63...|
 299603150| CC| [cc]| [cc]| (10000,[0],[1.0])|
 314287826| CC| [cc]| [cc]| (10000,[0],[1.0])|
 350216343| CC| [cc]| [cc]| (10000,[0],[1.0])|
 249764101|Liquidación IUVA ...|[liquidaci, n, iu...|[liquidaci, n, iu...|(10000,[0,9,10,32...|
 362400899| CC| [cc]| [cc]| (10000,[0],[1.0])|
+----------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [30]:
predictions = lrModel.transform(dataset1)


In [31]:
predictions.show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3940150610880117> in <module> 
 ----> 1 predictions . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 350 """
 351 if isinstance ( truncate , bool ) and truncate : 
 --> 352 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 353 else : 
 354 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o2636.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 638.0 failed 1 times, most recent failure: Lost task 0.0 in stage 638.0 (TID 5814, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (vector) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:620)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:49)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:126)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:125)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:87)
	at org.apache.spark.scheduler.Task.run(Task.scala:112)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:384)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: The columns of A don't match the number of elements of x. A: 9213, x: 10000
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.linalg.BLAS$.gemv(BLAS.scala:539)
	at org.apache.spark.ml.classification.LogisticRegressionModel$$anonfun$34.apply(LogisticRegression.scala:1007)
	at org.apache.spark.ml.classification.LogisticRegressionModel$$anonfun$34.apply(LogisticRegression.scala:1005)
	at org.apache.spark.ml.classification.LogisticRegressionModel.predictRaw(LogisticRegression.scala:1155)
	at org.apache.spark.ml.classification.LogisticRegressionModel.predictRaw(LogisticRegression.scala:930)
	at org.apache.spark.ml.classification.ProbabilisticClassificationModel$$anonfun$1.apply(ProbabilisticClassifier.scala:117)
	at org.apache.spark.ml.classification.ProbabilisticClassificationModel$$anonfun$1.apply(ProbabilisticClassifier.scala:116)
	... 12 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1747)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1735)
	at org.apache.spark.scheduler.DAGScheduler$

In [32]:
predictions.filter(predictions['prediction'] == 0) \
    .select("id_trn_ach","ref1","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o2838.select.
: org.apache.spark.sql.AnalysisException: cannot resolve '`label`' given input columns: [prediction, rawPrediction, probability, features, words, filtered, ref1, id_trn_ach];;
'Project [id_trn_ach#6278, ref1#6270, probability#7591, 'label, prediction#7599]
+- Filter (prediction#7599 = cast(0 as double))
 +- Project [id_trn_ach#6278, ref1#6270, words#7345, filtered#7350, features#7356, rawPrediction#7584, probability#7591, UDF(rawPrediction#7584) AS prediction#7599]
 +- Project [id_trn_ach#6278, ref1#6270, words#7345, filtered#7350, features#7356, rawPrediction#7584, UDF(rawPrediction#7584) AS probability#7591]
 +- Project [id_trn_ach#6278, ref1#6270, words#7345, filtered#7350, features#7356, UDF(features#7356) AS rawPrediction#7584]
 +- Project [id_trn_ach#6278, ref1#6270, words#7345, filtered#7350, features#7356]
 +- Project [id_trn_ach#6278, ref1#6270, sector#6274, words#7345, filtered#7350, UDF(filtered#7350) AS features#7356]
 +- Project [id_trn_ach#6278, ref1#6270, sector#6274, words#7345, UDF(words#7345) AS filtered#7350]
 +- Project [id_trn_ach#6278, ref1#6270, sector#6274, UDF(ref1#6270) AS words#7345]
 +- Filter isnull(sector#6274)
 +- Filter NOT (ref1#6270 = )
 +- Project [trim(cast(id_trn_ach#6181 as string), None) AS id_trn_ach#6278, ref1#6270, sector#6274]
 +- Project [id_trn_ach#6181, ref1#6270, trim(sector#6226, None) AS sector#6274]
 +- Project [id_trn_ach#6181, trim(ref1#6217, None) AS ref1#6270, sector#6226]
 +- Project [id_trn_ach#6181, ref1#6217, sector#6226]
 +- Project [id_trn_ach#6181, id_cliente#6190, fechahora#6199L, valor_trx#6208, ref1#6217, sector#6226, subsector#6235, _c7#6172 AS descripcion#6244]
 +- Project [id_trn_ach#6181, id_cliente#6190, fechahora#6199L, valor_trx#6208, ref1#6217, sector#6226, _c6#6171 AS subsector#6235, _c7#6172]
 +- Project [id_trn_ach#6181, id_cliente#6190, fechahora#6199L, valor_trx#6208, ref1#6217, _c5#6170 AS sector#6226, _c6#6171, _c7#6172]
 +- Project [id_trn_ach#6181, id_cliente#6190, fechahora#6199L, valor_trx#6208, _c4#6169 AS ref1#6217, _c5#6170, _c6#6171, _c7#6172]
 +- Project [id_trn_ach#6181, id_cliente#6190, fechahora#6199L, _c3#6168 AS valor_trx#6208, _c4#6169, _c5#6170, _c6#6171, _c7#6172]
 +- Project [id_trn_ach#6181, id_cliente#6190, _c2#6167L AS fechahora#6199L, _c3#6168, _c4#6169, _c5#6170, _c6#6171, _c7#6172]
 +- Project [id_trn_ach#6181, _c1#6166 AS id_cliente#6190, _c2#6167L, _c3#6168, _c4#6169, _c5#6170, _c6#6171, _c7#6172]
 +- Project [_c0#6165 AS id_trn_ach#6181, _c1#6166, _c2#6167L, _c3#6168, _c4#6169, _c5#6170, _c6#6171, _c7#6172]
 +- Relation[_c0#6165,_c1#6166,_c2#6167L,_c3#6168,_c4#6169,_c5#6170,_c6#6171,_c7#6172] csv

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$2.applyOrElse(CheckAnalysis.scala:112)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$2.applyOrElse(CheckAnalysis.scala:105)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:301)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:301)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:77)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.Quer